In [1]:
import numpy as np
import os
import warnings
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler


warnings.filterwarnings("ignore", category=FutureWarning)


cwd = os.getcwd()
tf.get_logger().setLevel('INFO')

In [2]:
#read data
X_test = pd.read_csv(cwd+r'\data\final\X_test.csv')
X_train = pd.read_csv(cwd+r'\data\final\X_train.csv')
X_val = pd.read_csv(cwd+r'\data\final\X_val.csv')
Y_test = pd.read_csv(cwd+r'\data\final\y_test.csv')
Y_train = pd.read_csv(cwd+r'\data\final\y_train.csv')
Y_val = pd.read_csv(cwd+r'\data\final\y_val.csv')

print(X_test.shape)
print(Y_test.shape)
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(6338, 32)
(6338, 1)
(29574, 32)
(29574, 1)
(6337, 32)
(6337, 1)


In [3]:
# total_awards_received	num_comments title (positve, negative, neutral), selftext (positve, negative, neutral), created (hour of day)
X_test['created'] = pd.to_datetime(X_test['created'])
X_train['created'] = pd.to_datetime(X_train['created'])
X_val['created'] = pd.to_datetime(X_val['created'])

# Create the new columns
X_test['hour_of_day'] = X_test['created'].dt.hour
X_test['day_of_week'] = X_test['created'].dt.day_name()

X_train['hour_of_day'] = X_train['created'].dt.hour
X_train['day_of_week'] = X_train['created'].dt.day_name()

X_val['hour_of_day'] = X_val['created'].dt.hour
X_val['day_of_week'] = X_val['created'].dt.day_name()

columns_to_retain = ['day_of_week', 'title', 'selftext', 'hour_of_day']

X_test = X_test[columns_to_retain]
X_train = X_train[columns_to_retain]
X_val = X_val[columns_to_retain]
Y_test = Y_test[['engagement_score']]
Y_train = Y_train[['engagement_score']]
Y_val = Y_val[['engagement_score']]

print(X_test.shape)
print(Y_test.shape)
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)


(6338, 4)
(6338, 1)
(29574, 4)
(29574, 1)
(6337, 4)
(6337, 1)


In [4]:

# Columns to standardize
cols_to_standardize_Y = ['engagement_score']

# Initialize the scaler
scaler = StandardScaler()

# Standardize the score
Y_train['engagement_score'] = scaler.fit_transform(Y_train[['engagement_score']])
Y_val['engagement_score'] = scaler.transform(Y_val[['engagement_score']])
Y_test['engagement_score'] = scaler.transform(Y_test[['engagement_score']])

# Convert specific columns to string
X_train['title'] = X_train['title'].astype(str)
X_train['selftext'] = X_train['selftext'].astype(str)

X_val['title'] = X_val['title'].astype(str)
X_val['selftext'] = X_val['selftext'].astype(str)

X_test['title'] = X_test['title'].astype(str)
X_test['selftext'] = X_test['selftext'].astype(str)

In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment intensity analyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\clift\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
def classify_sentiment(text):
    if not text:
        return 'neutral'
    sentiment_scores = sid.polarity_scores(text)
    compound = sentiment_scores['compound']
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [7]:
# Apply the function to classify sentiment
X_train['title'] = X_train['title'].apply(classify_sentiment)
X_train['selftext'] = X_train['selftext'].apply(classify_sentiment)

X_val['title'] = X_val['title'].apply(classify_sentiment)
X_val['selftext'] = X_val['selftext'].apply(classify_sentiment)

X_test['title'] = X_test['title'].apply(classify_sentiment)
X_test['selftext'] = X_test['selftext'].apply(classify_sentiment)

In [8]:
print(X_train.head(5))

  day_of_week     title  selftext  hour_of_day
0    Thursday  negative  positive           20
1    Thursday  positive  positive           12
2      Friday   neutral  positive           16
3   Wednesday   neutral  negative            7
4    Thursday   neutral  positive           17


In [9]:
# One-hot encode 'title'
X_train = pd.get_dummies(X_train, columns=['title'])
X_val = pd.get_dummies(X_val, columns=['title'])
X_test = pd.get_dummies(X_test, columns=['title'])
X_train = pd.get_dummies(X_train, columns=['selftext'])
X_val = pd.get_dummies(X_val, columns=['selftext'])
X_test = pd.get_dummies(X_test, columns=['selftext'])

In [10]:
X_train = pd.get_dummies(X_train, columns=['day_of_week'])
X_val = pd.get_dummies(X_val, columns=['day_of_week'])
X_test = pd.get_dummies(X_test, columns=['day_of_week'])

In [11]:
X_train = pd.get_dummies(X_train, columns=['hour_of_day'])
X_val = pd.get_dummies(X_val, columns=['hour_of_day'])
X_test = pd.get_dummies(X_test, columns=['hour_of_day'])

In [12]:
data_std = pd.concat([X_train, Y_train], axis=1)

corr = data_std.corr()
print(corr['engagement_score'].sort_values(ascending=False))

engagement_score         1.000000
hour_of_day_11           0.029030
hour_of_day_10           0.018544
hour_of_day_21           0.013102
selftext_negative        0.010098
title_neutral            0.006938
title_negative           0.006783
hour_of_day_12           0.006141
day_of_week_Thursday     0.005996
day_of_week_Tuesday      0.005474
hour_of_day_5            0.003964
hour_of_day_3            0.003640
hour_of_day_4            0.003152
hour_of_day_20           0.002023
day_of_week_Saturday     0.001806
hour_of_day_18           0.000483
day_of_week_Wednesday   -0.000324
day_of_week_Friday      -0.001358
hour_of_day_17          -0.001476
hour_of_day_13          -0.001530
hour_of_day_2           -0.002578
selftext_neutral        -0.002709
hour_of_day_15          -0.003206
hour_of_day_6           -0.003385
hour_of_day_9           -0.003852
day_of_week_Sunday      -0.004415
hour_of_day_1           -0.004612
hour_of_day_8           -0.004746
hour_of_day_22          -0.004839
hour_of_day_7 

In [13]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
def build_model(num_features, learning_rate):
    """Build a TF linear regression model using Keras."""
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)

    model = tf.keras.Sequential()
    model.add(Dense(
        units=1,        # output dim
        input_shape=[num_features],  # input dim
        use_bias=True,
        kernel_initializer=tf.keras.initializers.GlorotUniform(),  # Better weight initializer
        bias_initializer=tf.keras.initializers.Zeros(),  # Initialize bias to zero
        kernel_regularizer=l2(0.01)  # L2 regularization to prevent overfitting
    ))

    optimizer = SGD(learning_rate=learning_rate)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])

    return model

In [22]:
tf.random.set_seed(0)

# 2. Build and compile model
num_features = X_train.shape[1]
learning_rate = 0.001
model_tf = build_model(num_features, learning_rate)

# 3. Fit the model
num_epochs = 25
batch_size = 16

history = model_tf.fit(
    X_train, Y_train,
    epochs=num_epochs,
    batch_size=batch_size,
    validation_data=(X_val, Y_val)
)

# Evaluate the model
mse = model_tf.evaluate(X_test, Y_test)

Epoch 1/25


c:\Users\clift\anaconda\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1849/1849 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.1435 - mse: 1.1317 - val_loss: 0.7743 - val_mse: 0.7662
Epoch 2/25
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0928 - mse: 1.0856 - val_loss: 0.7536 - val_mse: 0.7482
Epoch 3/25
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0748 - mse: 1.0698 - val_loss: 0.7445 - val_mse: 0.7405
Epoch 4/25
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0663 - mse: 1.0626 - val_loss: 0.7396 - val_mse: 0.7366
Epoch 5/25
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0616 - mse: 1.0588 - val_loss: 0.7366 - val_mse: 0.7342
Epoch 6/25
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0587 - mse: 1.0565 - val_loss: 0.7347 - val_mse: 0.7327
Epoch 7/25
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0568 - mse: 1.0550 - val_loss: 0.7333 - val_mse: 0.7317
Epoch 8/25
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0555 - mse: 1.0540 - val_loss: 0.7323 - val_mse: 0.7309
Epoch 9/25
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/st

In [23]:
train_loss, train_MSE = model_tf.evaluate(X_train, Y_train, verbose=0)
val_loss, val_MSE = model_tf.evaluate(X_val, Y_val, verbose=0)
test_loss, test_MSE = model_tf.evaluate(X_test, Y_test, verbose=0)

print(f"MSE of train: {train_MSE:.4f}")
print(f"MSE of validation: {val_MSE:.4f}")
print(f"MSE of test: {test_MSE:.4f}")

MSE of train: 0.9980
MSE of validation: 0.7285
MSE of test: 0.3921
